In [3]:
!pip install gensim

In [4]:
import numpy as np 
import pandas as pd
import string
from tqdm import tqdm
import math

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import  hstack
from keras.layers import Input, concatenate,LSTM,Flatten, Dense,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint 
# from handFeatures import hand_features, clean, get_tokenized_lemmas, remove_stopwords
# from glove50_embedding import glove_sentence_embeddings

In [5]:
# run and tested
import os
import re
import nltk
import numpy as np
from sklearn import feature_extraction
from tqdm import tqdm


_wnl = nltk.WordNetLemmatizer()


def normalize_word(w):
    return _wnl.lemmatize(w).lower()


def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]


def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric
    
    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()


def remove_stopwords(l):
    # Removes stopwords from a list of tokens
    return [w for w in l if w not in feature_extraction.text.ENGLISH_STOP_WORDS]


def gen_or_load_feats(feat_fn, headlines, bodies, feature_file):
    if not os.path.isfile(feature_file):
        feats = feat_fn(headlines, bodies)
        np.save(feature_file, feats)

    return np.load(feature_file)




def word_overlap_features(features, headline, body):
    # common word/ total word
    clean_headline = clean(headline)
    clean_body = clean(body)
    clean_headline = get_tokenized_lemmas(clean_headline)
    clean_body = get_tokenized_lemmas(clean_body)
    feature = len(set(clean_headline).intersection(clean_body)) / float(len(set(clean_headline).union(clean_body)))
    features.append(feature)
    return features


def refuting_features(features,headline, body):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        # 'refute',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]
    X = []
    clean_headline = clean(headline)
    clean_headline = get_tokenized_lemmas(clean_headline)
    feature=0
    for wrd in clean_headline:
        if wrd in _refuting_words:
            feature=1
    features.append(feature)
    return features


def polarity_features(features,headline, body):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]

    def calculate_polarity(text):
        tokens = get_tokenized_lemmas(text)
        return sum([t in _refuting_words for t in tokens]) % 2
    X = []
    clean_headline = clean(headline)
    clean_body = clean(body)
    features.append(calculate_polarity(clean_headline))
    features.append(calculate_polarity(clean_body))
    return features

def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def chargrams(input, n):
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def append_chargrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in chargrams(" ".join(remove_stopwords(text_headline.split())), size)]
    grams_hits = 0
    grams_early_hits = 0
    grams_first_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
        if gram in text_body[:100]:
            grams_first_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    features.append(grams_first_hits)
    return features


def append_ngrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in ngrams(text_headline, size)]
    grams_hits = 0
    grams_early_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    return features


def hand_features(headlines, bodies):
    def binary_co_occurence(headline, body):
        # Count how many times a token in the title
        # appears in the body text.
        bin_count = 0
        bin_count_early = 0
        for headline_token in clean(headline).split(" "):
            if headline_token in clean(body):
                bin_count += 1
            if headline_token in clean(body)[:255]:
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def binary_co_occurence_stops(headline, body):
        # Count how many times a token in the title
        # appears in the body text. Stopwords in the title
        # are ignored.
        bin_count = 0
        bin_count_early = 0
        for headline_token in remove_stopwords(clean(headline).split(" ")):
            if headline_token in clean(body):
                bin_count += 1
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def count_grams(headline, body):
        # Count how many times an n-gram of the title
        # appears in the entire body, and intro paragraph

        clean_body = clean(body)
        clean_headline = clean(headline)
        features = []
        features = append_chargrams(features, clean_headline, clean_body, 2)
        features = append_chargrams(features, clean_headline, clean_body, 8)
        features = append_chargrams(features, clean_headline, clean_body, 4)
        features = append_chargrams(features, clean_headline, clean_body, 16)
        features = append_ngrams(features, clean_headline, clean_body, 2)
        features = append_ngrams(features, clean_headline, clean_body, 3)
        features = append_ngrams(features, clean_headline, clean_body, 4)
        features = append_ngrams(features, clean_headline, clean_body, 5)
        features = append_ngrams(features, clean_headline, clean_body, 6)
        return features

    def other_feature(headline, body):
        features=[]
        features= word_overlap_features(features, headline, body)
        features= refuting_features(features, headline, body)
        features= polarity_features(features,headline,body)
        return features
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        lst= binary_co_occurence(headline, body) + binary_co_occurence_stops(headline, body) + count_grams(headline, body)+ other_feature(headline, body)
        X.append(lst)


    X =np.array(X)
    return X




# x= hand_features(h,t)



In [6]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    
#matrix1 = confusion_matrix(test_labels, prediction)
#plot_confusion_matrix(cm=matrix1,target_names=['agree', 'disagree', 'discuss', 'unrelated'])


In [7]:
import string
import numpy as np
from tqdm import tqdm
from nltk.corpus import stopwords

# Ensure nltk stopwords are downloaded
import nltk
nltk.download('stopwords')

# Load GloVe embeddings
filename = '../input/glove.6B.100d.txt'
glove = {}
with open(filename, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        glove[word] = coefs

def pre_process(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords and split text
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return text

def make_embed(text):
    feature = np.zeros((len(text), 100), dtype='object')
    c = 0
    for wrd in text:
        if wrd in glove:
            feature[c] = glove[wrd].reshape(1, 100)
            c += 1
    res = np.mean(feature, axis=0)
    return res

def glove_sentence_embeddings(texts):
    embed = []
    for i in tqdm(range(len(texts))):
        clean_text = pre_process(texts[i])
        embed.append(make_embed(clean_text))
    return np.array(embed, dtype='float')

# Example usage
texts = [
    "This is a sample sentence.",
    "Another example sentence for embedding."
]
embeddings = glove_sentence_embeddings(texts)
print(embeddings)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\visha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 2/2 [00:00<00:00, 306.57it/s]

[[ 0.154305    0.40436     0.0659825  -0.15497201  0.20514999  0.80122498
   0.29861498  0.65231    -0.55372001  0.77741    -0.59126     0.60309999
  -0.35745649  0.13796     0.539635    0.33880549 -0.14214999 -0.31540001
  -0.08566999  0.16207501 -0.044045   -0.532755   -0.03116    -0.04241499
  -0.14816999  0.17823501  0.31223399 -0.4232837   0.108767    0.26135001
   0.449995   -0.18367199 -0.136246   -0.033645    0.28059     0.189042
   0.22823501 -0.06801499 -0.23754101 -0.42772001 -0.245415   -0.45503999
   0.360415   -0.20957001  0.01927     0.15212     0.14541501 -1.17711997
  -0.114215   -0.232645    1.04551497 -0.37946998  0.39599499  0.76536503
  -0.58912998 -0.89366499 -0.4055775  -0.36572001  1.60865003  0.29840901
   0.0815      0.14972501 -0.08866999 -0.25362504  1.05665499 -0.00167999
   0.34857501  0.20403001 -0.53334001  0.13069999  0.022135    0.554295
   0.57542001  0.422685    0.251855    0.71860501 -0.059782   -0.88275999
  -0.36005999 -0.37763     0.30704501 -0.1

In [8]:
trainFile=pd.read_csv('../input/train_merged.csv',encoding='latin-1')
testFile=pd.read_csv('../input/test_merged.csv', encoding='latin-1')

In [9]:
trainBody= trainFile['articleBody'].tolist()
trainHead= trainFile['Headline'].tolist()
testBody= testFile['articleBody'].tolist()
testHead= testFile['Headline'].tolist()


In [10]:
train_handF= hand_features(trainHead, trainBody)
test_handF=hand_features(testHead, testBody)

0it [00:00, ?it/s]

49972it [10:01, 83.01it/s] 
25413it [04:45, 88.95it/s] 


In [11]:
trainBodyFeat= glove_sentence_embeddings(trainBody)
trainHeadFeat= glove_sentence_embeddings(trainHead)
testBodyFeat= glove_sentence_embeddings(testBody)
testHeadFeat= glove_sentence_embeddings(testHead)

100%|██████████| 25413/25413 [01:36<00:00, 263.81it/s]


In [12]:
np.save('glove100_trainBodyFeat',trainBodyFeat)
np.save('glove100_trainHeadFeat',trainHeadFeat)
np.save('glove100_testBodyFeat',testBodyFeat)
np.save('glove100_testHeadFeat',testHeadFeat)

In [13]:
trainFeat= np.concatenate((trainHeadFeat, trainBodyFeat, train_handF), axis=-1)
testFeat= np.concatenate((testHeadFeat, testBodyFeat, test_handF), axis=-1)

In [14]:
np.save('trainFeatMatrix_glove_100',trainFeat)
np.save('testFeatMatrix_glove_100',testFeat)

In [15]:
print(trainFeat.shape,testFeat.shape)

(49972, 230) (25413, 230)


In [16]:
train_labels= trainFile['Stance'].copy()
test_labels= testFile['Stance'].copy()

In [17]:
inp = Input(shape=(trainFeat.shape[1],))
lay1= Dense(200, activation= 'relu')(inp)
lay1= Dropout(0.3)(lay1)
lay2= Dense(80, activation= 'relu')(lay1)
outp= Dense(4,activation='sigmoid')(lay2)
model= Model(inputs=[inp], outputs=[outp])
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 230)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │        46,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 80)             │        16,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           324 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,604 (244.55 KB)

 Trainable params: 62,604 (244.55 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model_json = model.to_json()
with open("model_FNC.json", "w") as json_file:
    json_file.write(model_json)

In [19]:
filepath=r"weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5.keras"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

In [20]:
trainY = pd.get_dummies(trainFile['Stance']).values
testY = pd.get_dummies(testFile['Stance'] ).values

In [21]:
model.fit([trainFeat],[trainY],epochs=5,batch_size=13,verbose=1, validation_split=0.1, callbacks=callbacks_list, shuffle=True)

Epoch 1/5
3460/3460 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.8624 - loss: 0.3837 - val_accuracy: 0.8333 - val_loss: 0.4478
Epoch 2/5
  73/3460 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.8880 - loss: 0.3295

c:\Users\visha\anaconda3\envs\pyenv\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


3460/3460 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8894 - loss: 0.3029 - val_accuracy: 0.8407 - val_loss: 0.4338
Epoch 3/5
3460/3460 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8959 - loss: 0.2812 - val_accuracy: 0.8405 - val_loss: 0.4391
Epoch 4/5
3460/3460 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8983 - loss: 0.2739 - val_accuracy: 0.8543 - val_loss: 0.4390
Epoch 5/5
3460/3460 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9036 - loss: 0.2576 - val_accuracy: 0.8533 - val_loss: 0.4183


In [22]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
tok= Tokenizer()
model_json = model.to_json()
with open("../output/glove/fakenews.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("../output/glove/fakenews.weights.h5")

with open('../output/glove/fakenews.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [23]:
scor, acc = model.evaluate([testFeat],[testY])

795/795 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9003 - loss: 0.3188


In [24]:
acc,scor

(0.8532640933990479, 0.4865679442882538)